In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

In [205]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

all_teams = []

base_url = 'https://fbref.com'

# Get Premier League main page
html = requests.get('https://fbref.com/en/comps/9/Premier-League-Stats').text
soup = BeautifulSoup(html, 'lxml')

# Get first table containing teams
table = soup.find_all('table', class_='stats_table')[0]

# Parse rows to get team id and name together
rows = table.find_all('tr')
for row in rows:
    a_tag = row.find('a')
    if not a_tag:
        continue
    href = a_tag.get('href')
    if href and '/squads/' in href:
        parts = href.split('/')
        if len(parts) < 5:
            continue
        team_id = parts[3]
        team_name_raw = parts[-1]
        team_name = team_name_raw.replace('-Stats', '')
        
        # Construct team URL for 2024–2025 season
        team_url = f"{base_url}/en/squads/{team_id}/2024-2025/{team_name}-Stats"
        print(f"Scraping {team_url}")
        
        # Get team page
        data = requests.get(team_url).text
        soup_team = BeautifulSoup(data, 'lxml')

        # Get all stats tables on the page
        stats_tables = soup_team.find_all('table', class_='stats_table')
        if not stats_tables:
            print(f"No stats table found for {team_url}")
            continue

        stats_table = stats_tables[0]

        # Remove <tfoot> section (which contains "Squad Total", "Opponent Total")
        tfoot = stats_table.find('tfoot')
        if tfoot:
            tfoot.decompose()  # This removes it from the BeautifulSoup tree

        # Parse cleaned table using pandas
        filtered_table_html = str(stats_table)
        team_data = pd.read_html(filtered_table_html)[0]
        team_data['Team'] = team_name
        all_teams.append(team_data)

        time.sleep(5)  # Be nice to the server

# Combine all team data into one DataFrame
stat_df = pd.concat(all_teams, ignore_index=True)


Scraping https://fbref.com/en/squads/18bb7c10/2024-2025/Arsenal-Stats
Scraping https://fbref.com/en/squads/8602292d/2024-2025/Aston-Villa-Stats
Scraping https://fbref.com/en/squads/4ba7cbea/2024-2025/Bournemouth-Stats
Scraping https://fbref.com/en/squads/cd051869/2024-2025/Brentford-Stats
Scraping https://fbref.com/en/squads/d07537b9/2024-2025/Brighton-and-Hove-Albion-Stats
Scraping https://fbref.com/en/squads/943e8050/2024-2025/Burnley-Stats
Scraping https://fbref.com/en/squads/cff3d9bb/2024-2025/Chelsea-Stats
Scraping https://fbref.com/en/squads/47c64c55/2024-2025/Crystal-Palace-Stats
Scraping https://fbref.com/en/squads/d3fd31cc/2024-2025/Everton-Stats
Scraping https://fbref.com/en/squads/fd962109/2024-2025/Fulham-Stats
Scraping https://fbref.com/en/squads/5bfb9659/2024-2025/Leeds-United-Stats
Scraping https://fbref.com/en/squads/822bd0ba/2024-2025/Liverpool-Stats
Scraping https://fbref.com/en/squads/b8fd03ef/2024-2025/Manchester-City-Stats
Scraping https://fbref.com/en/squads/19538

In [206]:
stat_df['Team'].unique()

array(['Arsenal', 'Aston-Villa', 'Bournemouth', 'Brentford',
       'Brighton-and-Hove-Albion', 'Burnley', 'Chelsea', 'Crystal-Palace',
       'Everton', 'Fulham', 'Leeds-United', 'Liverpool',
       'Manchester-City', 'Manchester-United', 'Newcastle-United',
       'Nottingham-Forest', 'Sunderland', 'Tottenham-Hotspur',
       'West-Ham-United', 'Wolverhampton-Wanderers'], dtype=object)

In [207]:
stat_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in stat_df.columns]

In [208]:
stat_df.rename(columns={
    'Unnamed: 0_level_0_Player': 'Player Name',
    'Unnamed: 1_level_0_Nation': 'Nation',
    'Unnamed: 2_level_0_Pos': 'Position',
    'Unnamed: 3_level_0_Age': 'Age',
    'Unnamed: 4_level_0_MP': 'Matches Played'
}, inplace=True)

In [209]:
stat_df.loc[stat_df['Team_'].isin(['Brentford', 'Fulham']), 'Matches Played'] = stat_df.loc[stat_df['Team_'].isin(['Brentford', 'Fulham']), 'Playing Time_MP']

In [211]:
stat_df[stat_df['Team_'] == 'Brentford']

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Unnamed: 33_level_0_Matches,Team_,Playing Time_MP
110,Nathan Collins,ie IRL,DF,23.0,38.0,38,3420.0,38.0,2.0,3.0,...,0.05,0.13,0.07,0.05,0.11,0.07,0.11,Matches,Brentford,38.0
111,Bryan Mbeumo,cm CMR,FW,24.0,38.0,38,3414.0,37.9,20.0,7.0,...,0.40,0.58,0.32,0.22,0.54,0.20,0.42,Matches,Brentford,38.0
112,Mark Flekken,nl NED,GK,31.0,37.0,37,3275.0,36.4,0.0,2.0,...,0.00,0.05,0.00,0.01,0.01,0.00,0.01,Matches,Brentford,37.0
113,Keane Lewis-Potter,eng ENG,"DF,FW",23.0,38.0,36,3093.0,34.4,1.0,3.0,...,0.03,0.12,0.09,0.09,0.18,0.09,0.18,Matches,Brentford,38.0
114,Yoane Wissa,cd COD,FW,27.0,35.0,34,2919.0,32.4,19.0,4.0,...,0.59,0.71,0.57,0.08,0.65,0.57,0.65,Matches,Brentford,35.0
115,Mikkel Damsgaard,dk DEN,"MF,FW",24.0,38.0,34,2911.0,32.3,2.0,10.0,...,0.06,0.37,0.09,0.26,0.35,0.09,0.35,Matches,Brentford,38.0
116,Christian Nørgaard,dk DEN,MF,30.0,34.0,34,2819.0,31.3,5.0,4.0,...,0.16,0.29,0.14,0.04,0.18,0.14,0.18,Matches,Brentford,34.0
117,Sepp van den Berg,nl NED,DF,22.0,31.0,29,2586.0,28.7,0.0,0.0,...,0.00,0.00,0.07,0.03,0.10,0.07,0.10,Matches,Brentford,31.0
118,Vitaly Janelt,de GER,MF,26.0,32.0,27,2254.0,25.0,1.0,3.0,...,0.04,0.16,0.03,0.09,0.12,0.03,0.12,Matches,Brentford,32.0
119,Kevin Schade,de GER,FW,22.0,38.0,26,2293.0,25.5,11.0,2.0,...,0.43,0.51,0.32,0.14,0.47,0.32,0.47,Matches,Brentford,38.0


In [212]:
print(stat_df.isna().sum())

Player Name                      0
Nation                           1
Position                         0
Age                              1
Matches Played                   0
Playing Time_Starts              0
Playing Time_Min               137
Playing Time_90s               137
Performance_Gls                137
Performance_Ast                137
Performance_G+A                137
Performance_G-PK               137
Performance_PK                 137
Performance_PKatt              137
Performance_CrdY               137
Performance_CrdR               137
Expected_xG                    137
Expected_npxG                  137
Expected_xAG                   137
Expected_npxG+xAG              137
Progression_PrgC               137
Progression_PrgP               137
Progression_PrgR               137
Per 90 Minutes_Gls             137
Per 90 Minutes_Ast             137
Per 90 Minutes_G+A             137
Per 90 Minutes_G-PK            137
Per 90 Minutes_G+A-PK          137
Per 90 Minutes_xG   

In [214]:
stat_df[stat_df['Nation'].isna()]

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Unnamed: 33_level_0_Matches,Team_,Playing Time_MP
698,Mateus Mane,NaN,MF,NaN,1.0,0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Matches,Wolverhampton-Wanderers,NaN


In [215]:
stat_df.drop(columns=["Playing Time_MP"], inplace=True)

In [216]:
stat_df['Team_'].unique()

array(['Arsenal', 'Aston-Villa', 'Bournemouth', 'Brentford',
       'Brighton-and-Hove-Albion', 'Burnley', 'Chelsea', 'Crystal-Palace',
       'Everton', 'Fulham', 'Leeds-United', 'Liverpool',
       'Manchester-City', 'Manchester-United', 'Newcastle-United',
       'Nottingham-Forest', 'Sunderland', 'Tottenham-Hotspur',
       'West-Ham-United', 'Wolverhampton-Wanderers'], dtype=object)

In [217]:
print(stat_df.isna().sum())

Player Name                      0
Nation                           1
Position                         0
Age                              1
Matches Played                   0
Playing Time_Starts              0
Playing Time_Min               137
Playing Time_90s               137
Performance_Gls                137
Performance_Ast                137
Performance_G+A                137
Performance_G-PK               137
Performance_PK                 137
Performance_PKatt              137
Performance_CrdY               137
Performance_CrdR               137
Expected_xG                    137
Expected_npxG                  137
Expected_xAG                   137
Expected_npxG+xAG              137
Progression_PrgC               137
Progression_PrgP               137
Progression_PrgR               137
Per 90 Minutes_Gls             137
Per 90 Minutes_Ast             137
Per 90 Minutes_G+A             137
Per 90 Minutes_G-PK            137
Per 90 Minutes_G+A-PK          137
Per 90 Minutes_xG   

In [218]:
stat_df.dropna(inplace=True)

In [219]:
stat_df['Team_'].unique()

array(['Arsenal', 'Aston-Villa', 'Bournemouth', 'Brentford',
       'Brighton-and-Hove-Albion', 'Burnley', 'Chelsea', 'Crystal-Palace',
       'Everton', 'Fulham', 'Leeds-United', 'Liverpool',
       'Manchester-City', 'Manchester-United', 'Newcastle-United',
       'Nottingham-Forest', 'Sunderland', 'Tottenham-Hotspur',
       'West-Ham-United', 'Wolverhampton-Wanderers'], dtype=object)

In [220]:
stat_df

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Unnamed: 33_level_0_Matches,Team_
0,David Raya,es ESP,GK,28.0,38.0,38,3420.0,38.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,Arsenal
1,William Saliba,fr FRA,DF,23.0,35.0,35,3039.0,33.8,2.0,0.0,...,0.06,0.06,0.06,0.07,0.03,0.09,0.07,0.09,Matches,Arsenal
2,Declan Rice,eng ENG,MF,25.0,35.0,33,2825.0,31.4,4.0,7.0,...,0.35,0.13,0.35,0.11,0.21,0.32,0.11,0.32,Matches,Arsenal
3,Thomas Partey,gh GHA,"MF,DF",31.0,35.0,31,2797.0,31.1,4.0,2.0,...,0.19,0.13,0.19,0.07,0.07,0.14,0.07,0.14,Matches,Arsenal
4,Leandro Trossard,be BEL,FW,29.0,38.0,28,2546.0,28.3,8.0,7.0,...,0.53,0.28,0.53,0.25,0.22,0.47,0.25,0.47,Matches,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Daniel Podence,pt POR,"MF,FW",28.0,2.0,0,48.0,0.5,0.0,0.0,...,0.00,0.00,0.00,0.07,0.00,0.07,0.07,0.07,Matches,Wolverhampton-Wanderers
697,Chiquinho,pt POR,MF,24.0,1.0,0,7.0,0.1,0.0,0.0,...,0.00,0.00,0.00,0.00,0.42,0.42,0.00,0.42,Matches,Wolverhampton-Wanderers
699,Tom King,wls WAL,GK,29.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,Wolverhampton-Wanderers
700,Alfie Pond,eng ENG,DF,20.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,Wolverhampton-Wanderers


In [221]:
stat_df.drop(columns=["Unnamed: 33_level_0_Matches"], inplace=True)

In [222]:
stat_df['League'] = 'English Premier League'

In [223]:
non_epl_teams = ['Sunderland', 'Burnley', 'Leeds-United']
stat_df.loc[stat_df['Team_'].isin(non_epl_teams), 'League'] = 'EFL Championship'

In [224]:
stat_df.rename(columns = {
    'Team_': 'Team'
}, inplace=True)

In [225]:
stat_df.rename(columns = {
    'League': 'League_Previous_Season'
}, inplace=True)

In [226]:
stat_df

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Team,League_Previous_Season
0,David Raya,es ESP,GK,28.0,38.0,38,3420.0,38.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Arsenal,English Premier League
1,William Saliba,fr FRA,DF,23.0,35.0,35,3039.0,33.8,2.0,0.0,...,0.06,0.06,0.06,0.07,0.03,0.09,0.07,0.09,Arsenal,English Premier League
2,Declan Rice,eng ENG,MF,25.0,35.0,33,2825.0,31.4,4.0,7.0,...,0.35,0.13,0.35,0.11,0.21,0.32,0.11,0.32,Arsenal,English Premier League
3,Thomas Partey,gh GHA,"MF,DF",31.0,35.0,31,2797.0,31.1,4.0,2.0,...,0.19,0.13,0.19,0.07,0.07,0.14,0.07,0.14,Arsenal,English Premier League
4,Leandro Trossard,be BEL,FW,29.0,38.0,28,2546.0,28.3,8.0,7.0,...,0.53,0.28,0.53,0.25,0.22,0.47,0.25,0.47,Arsenal,English Premier League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Daniel Podence,pt POR,"MF,FW",28.0,2.0,0,48.0,0.5,0.0,0.0,...,0.00,0.00,0.00,0.07,0.00,0.07,0.07,0.07,Wolverhampton-Wanderers,English Premier League
697,Chiquinho,pt POR,MF,24.0,1.0,0,7.0,0.1,0.0,0.0,...,0.00,0.00,0.00,0.00,0.42,0.42,0.00,0.42,Wolverhampton-Wanderers,English Premier League
699,Tom King,wls WAL,GK,29.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Wolverhampton-Wanderers,English Premier League
700,Alfie Pond,eng ENG,DF,20.0,1.0,0,1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Wolverhampton-Wanderers,English Premier League


In [235]:
stat_df[(stat_df['Age'] <= 19) & (stat_df['Matches Played'] >= 10)]

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Team,League_Previous_Season
12,Myles Lewis-Skelly,eng ENG,DF,17.0,23.0,15,1369.0,15.2,1.0,0.0,...,0.07,0.07,0.07,0.01,0.03,0.04,0.01,0.04,Arsenal,English Premier League
15,Ethan Nwaneri,eng ENG,"FW,MF",17.0,26.0,11,895.0,9.9,4.0,2.0,...,0.60,0.40,0.60,0.12,0.12,0.24,0.12,0.24,Arsenal,English Premier League
80,Dean Huijsen,es ESP,DF,19.0,32.0,26,2427.0,27.0,3.0,2.0,...,0.19,0.11,0.19,0.07,0.08,0.14,0.07,0.14,Bournemouth,English Premier League
132,Yunus Emre Konak,tr TUR,"FW,MF",18.0,10.0,0,42.0,0.5,0.0,0.0,...,0.00,0.00,0.00,0.33,0.00,0.33,0.33,0.33,Brentford,English Premier League
154,Jack Hinshelwood,eng ENG,"MF,DF",19.0,26.0,22,1842.0,20.5,5.0,2.0,...,0.34,0.24,0.34,0.13,0.07,0.21,0.13,0.21,Brighton-and-Hove-Albion,English Premier League
169,Evan Ferguson,ie IRL,FW,19.0,13.0,2,246.0,2.7,1.0,0.0,...,0.37,0.37,0.37,0.20,0.15,0.35,0.20,0.35,Brighton-and-Hove-Albion,English Premier League
199,Luca Koleosho,it ITA,"FW,MF",19.0,28.0,20,1677.0,18.6,2.0,0.0,...,0.11,0.11,0.11,0.10,0.13,0.23,0.10,0.23,Burnley,EFL Championship
435,Rico Lewis,eng ENG,"DF,MF",19.0,28.0,21,1893.0,21.0,1.0,2.0,...,0.14,0.05,0.14,0.07,0.02,0.08,0.07,0.08,Manchester-City,English Premier League
474,Kobbie Mainoo,eng ENG,MF,19.0,25.0,19,1651.0,18.3,0.0,0.0,...,0.00,0.00,0.00,0.04,0.09,0.12,0.04,0.12,Manchester-United,English Premier League
477,Leny Yoro,fr FRA,DF,18.0,21.0,12,1165.0,12.9,0.0,0.0,...,0.00,0.00,0.00,0.05,0.01,0.06,0.05,0.06,Manchester-United,English Premier League


In [150]:
stat_df.to_csv("epl_stats.csv")